In [1]:
#from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels import regression,stats
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
from collections import deque

In [2]:
from os import listdir
from os.path import isfile, join
from collections import defaultdict

mypath = 'eric_jh_data/'
countries = ['Australia', 'Japan', 'China']
adr_dict = {}
return_dict = defaultdict(list)
fx_dict = {'Australia':('eric_jh_data/Forex/AUD_USD.csv',0),
           'Japan':('eric_jh_data/Forex/USD_JPY.csv',1),
           'China':('eric_jh_data/Forex/USD_HKD.csv',1)}
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    adr_dict[country] = adr_names

In [42]:
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    adr_dict[country] = adr_names
    
    for adr in adr_names:
        print(data_processing(country, adr, fx_dict))

0.9968998155966732
no data for underlying data of PLL_PLL from Australia
None
0.0016854872904929566
no data for underlying data of ATHE_ATH from Australia
None
no data for underlying data of IMMP_IMM from Australia
None
0.9999189577547546
no data for underlying data of KZIA_KZA from Australia
None
0.19828354680499435
0.025514770969943097
0.999506635151424
0.9998565489650568
5.00274185269611
0.19973421064724692
5.0009639745568455
2.0007728095562984
1.0005206886504754
1.000465778069457
0.9991914463657956
0.4997965869827934
0.019984795881583636
0.07702157139299512
0.2000834607772626
0.020005056415712753
0.009995426701454152
0.009994709455041776
0.04004598282939929
0.025057346730456967
0.009995608807595648


In [21]:
sorted(adr_dict["China"])

['ACH_2600',
 'BGNE_6160',
 'CEA_670',
 'HNP_902',
 'LFC_2628',
 'PTR_857',
 'SHI_338',
 'SNP_386',
 'ZNH_1055']

In [6]:
data_processing("China", 'CEA_670', fx_dict)

,date,adr_open,adr_close,stock_open,stock_close,fx_open,fx_close,stock_open_usd,stock_close_usd
0,2015-04-13,39.65,38.93,5.91,5.91,7.75006,7.75013,38.128737,38.128393
1,2015-04-14,38.00,37.63,5.91,5.91,7.75077,7.75084,38.125244,38.124900
2,2015-04-15,37.35,37.40,5.91,5.91,7.75150,7.75159,38.121654,38.121211
3,2015-04-16,37.09,37.61,5.91,5.91,7.75151,7.75163,38.121605,38.121015
4,2015-04-17,36.96,37.45,5.91,5.91,7.75156,7.75163,38.121359,38.121015
...,...,...,...,...,...,...,...,...,...
1436,2021-03-29,23.96,24.06,3.70,3.71,7.77221,7.77226,23.802754,23.866932
1437,2021-03-30,23.78,23.83,3.71,3.74,7.77439,7.77445,23.860393,24.053148
1438,2021-03-31,23.80,23.40,3.71,3.66,7.77445,7.77451,23.860209,23.538461
1439,2021-04-01,23.48,23.56,3.67,3.65,7.77517,7.77521,23.600770,23.472035


In [12]:
country = "Australia"
adr = 'WBK_WBC'
adr_path = f'eric_jh_data/{country}/{adr}/adr.csv'
stock_path =  f'eric_jh_data/{country}/{adr}/underlying.csv'
fx_path = fx_dict[country][0]
fx_type =  fx_dict[country][1]

In [13]:
adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})

In [25]:
fx_df = pd.read_csv(fx_path, index_col = 0).rename(columns = {'close':'fx_close', 'open':'fx_open'})
merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
merged_df = pd.merge(merged_df, fx_df.loc[:,['date', 'fx_open','fx_close']])


In [26]:
if fx_type == 1:
    merged_df['stock_open_usd'] = merged_df['stock_open']/merged_df['fx_open']
    merged_df['stock_close_usd'] = merged_df['stock_close']/merged_df['fx_close']
else:
    merged_df['stock_open_usd'] = merged_df['stock_open']*merged_df['fx_open']
    merged_df['stock_close_usd'] = merged_df['stock_close']*merged_df['fx_close']

In [27]:
ratio =round(merged_df.loc[1,'stock_close_usd']/merged_df.loc[1,'adr_open'])

In [31]:
np.mean(merged_df.loc[:,'stock_close_usd']/merged_df.loc[:,'adr_open'])

0.9999189577547546

In [29]:
merged_df.loc[1,'stock_close_usd']/merged_df.loc[1,'adr_open']

0.996073642384106

In [ ]:
adr_path = f'eric_jh_data/{country}/{adr}/adr.csv'
stock_path =  f'eric_jh_data/{country}/{adr}/underlying.csv'
fx_path = fx_dict[country][0]
fx_type =  fx_dict[country][1]
try:
    adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
except:
    print(f"no data for ADR data of {adr} from {country}")
    return None
try:
    stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
except:
    print(f"no data for underlying data of {adr} from {country}")
    return None
fx_df = pd.read_csv(fx_path, index_col = 0).rename(columns = {'close':'fx_close', 'open':'fx_open'})

merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
merged_df = pd.merge(merged_df, fx_df.loc[:,['date', 'fx_open','fx_close']])

if fx_type == 1:
    merged_df['stock_open_usd'] = merged_df['stock_open']/merged_df['fx_open']
    merged_df['stock_close_usd'] = merged_df['stock_close']/merged_df['fx_close']
else:
    merged_df['stock_open_usd'] = merged_df['stock_open']*merged_df['fx_open']
    merged_df['stock_close_usd'] = merged_df['stock_close']*merged_df['fx_close']

ratio =round(merged_df.loc[1,'stock_close_usd']/merged_df.loc[1,'adr_open'])
if ratio >= 1:
    merged_df['adr_open'] = merged_df['adr_open']*ratio
    merged_df['adr_close'] = merged_df['adr_close']*ratio
else:
    ratio = round(merged_df.loc[1,'adr_open']/merged_df.loc[1,'stock_close_usd'])
    merged_df['stock_open_usd'] = merged_df['stock_open_usd']*ratio
    merged_df['stock_close_usd'] = merged_df['stock_close_usd']*ratio

return merged_df

In [91]:
from os import listdir
from os.path import isfile, join
from collections import defaultdict

mypath = 'eric_jh_data/'
countries = ['Australia', 'Japan', 'China']
adr_dict = {}
return_dict = defaultdict(list)
fx_dict = {'Australia':('eric_jh_data/Forex/AUD_USD.csv',0),
           'Japan':('eric_jh_data/Forex/USD_JPY.csv',1),
           'China':('eric_jh_data/Forex/USD_HKD.csv',1)}
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    adr_dict[country] = adr_names
    
    for adr in adr_names:
        merged_df = data_processing(country, adr, fx_dict)
        if isinstance(merged_df, pd.core.frame.DataFrame):
            ret, trade_records = pairs_trade(merged_df)
            return_dict[country].append([adr, ret])
            logs = [f'The return of ADR_underlying pairs trading for {adr} from {country} is {ret*100}%\n']
            logs = logs + trade_records 
            
            
            fname = f'eric_jh_data/{country}/{adr}/logs.txt'
            f = open(fname, 'w')
            f.writelines(logs)
            f.close()
            
        
        

There is no underlying data for Australia, PLL_PLL
There is no underlying data for Australia, ATHE_ATH
There is no underlying data for Australia, IMMP_IMM
There is no underlying data for Australia, KZIA_KZA


In [41]:
def data_processing(country, adr, fx_dict):
    adr_path = f'eric_jh_data/{country}/{adr}/adr.csv'
    stock_path =  f'eric_jh_data/{country}/{adr}/underlying.csv'
    fx_path = fx_dict[country][0]
    fx_type =  fx_dict[country][1]
    try:
        adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
    except:
        print(f"no data for ADR data of {adr} from {country}")
        return None
    try:
        stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
    except:
        print(f"no data for underlying data of {adr} from {country}")
        return None
    fx_df = pd.read_csv(fx_path, index_col = 0).rename(columns = {'close':'fx_close', 'open':'fx_open'})

    merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
    merged_df = pd.merge(merged_df, fx_df.loc[:,['date', 'fx_open','fx_close']])

    if fx_type == 1:
        merged_df['stock_open_usd'] = merged_df['stock_open']/merged_df['fx_open']
        merged_df['stock_close_usd'] = merged_df['stock_close']/merged_df['fx_close']
    else:
        merged_df['stock_open_usd'] = merged_df['stock_open']*merged_df['fx_open']
        merged_df['stock_close_usd'] = merged_df['stock_close']*merged_df['fx_close']

    ratio = np.mean(merged_df['stock_close_usd']/merged_df['adr_open'])
    return ratio
    if ratio >= 1:
        merged_df['adr_open'] = merged_df['adr_open']*ratio
        merged_df['adr_close'] = merged_df['adr_close']*ratio
    else:
        ratio = round(merged_df.loc[1,'adr_open']/merged_df.loc[1,'stock_close_usd'])
        merged_df['stock_open_usd'] = merged_df['stock_open_usd']*ratio
        merged_df['stock_close_usd'] = merged_df['stock_close_usd']*ratio
        
    return merged_df
    

In [130]:
# Because 1 ADR is not necessarily 1 Underlying, we find the ratio to convert the two..
ratio =round(merged_df.loc[1,'stock_close_usd']/merged_df.loc[1,'adr_open'])
if ratio >= 1:
    merged_df['adr_open'] = merged_df['adr_open']*ratio
    merged_df['adr_close'] = merged_df['adr_close']*ratio
else:
    ratio = round(merged_df.loc[1,'adr_open']/merged_df.loc[1,'stock_close_usd'])
    merged_df['stock_open_usd'] = merged_df['stock_open_usd']*ratio
    merged_df['stock_close_usd'] = merged_df['stock_close_usd']*ratio

In [43]:
merged_df[]

,date,adr_open,adr_close,stock_open,stock_close,fx_open,fx_close,stock_open_usd,stock_close_usd
0,2015-04-13,30.36,30.13,39.99,39.77,0.76009,0.76013,30.395999,30.230370
1,2015-04-14,30.20,30.36,39.70,39.56,0.76036,0.76040,30.186292,30.081424
2,2015-04-15,29.93,30.09,39.40,38.92,0.76161,0.76165,30.007434,29.643418
3,2015-04-16,30.48,30.60,39.34,39.27,0.77584,0.77588,30.521546,30.468808
4,2015-04-17,30.28,30.07,39.33,38.90,0.77896,0.77901,30.636497,30.303489
...,...,...,...,...,...,...,...,...,...
1475,2021-03-30,18.55,18.61,24.45,24.36,0.76240,0.76242,18.640680,18.572551
1476,2021-03-31,18.61,18.56,24.45,24.41,0.76080,0.76082,18.601560,18.571616
1477,2021-04-01,18.61,18.73,24.42,24.54,0.75840,0.75841,18.520128,18.611381
1478,2021-04-06,18.82,18.84,24.75,24.70,0.76446,0.76449,18.920385,18.882903


In [57]:
"2016" <= merged_df["date"] <= "2021"

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [87]:
test = ['a\n','b\n','c\n']
fname = 'text.txt'
f = open(fname, "w")
f.writelines(test)
f.close()

In [89]:
"""
Naive strategy for testing on the SONY adr and stock pair
To open a position, we check the CLOSE price of adr, compared it to CLOSE px of 
stock. We buy the stock on the next trading next OPEN for Asian market

To close a position,  we check the CLOSE price of adr, compared it to CLOSE px of 
stock. We sell the stock on the next trading next OPEN for Asian market
"""
def pairs_trade(merged_df, lookback = 100, cash = 100000):

    starting_cash = cash
    stock_pos, adr_pos = 0, 0
    diff_record = deque(maxlen = lookback)
    trade_records = []


    for index, row in merged_df.iterrows():
        # check if there is a px diff between close and stock_close effective
        # If index < lookback, we do not place any trade
        if index < lookback:
            diff_record.append(row['adr_close'] - row['stock_close_usd'])
            pass

        # If we have passed the initial lookback window
        # enter the position if diff is significant
        if row['adr_close'] - row['stock_close_usd'] > np.array(diff_record).mean() + np.array(diff_record).std():
            if stock_pos == 0 and adr_pos == 0:
                quantity = min(int(0.5*cash/row['adr_close']),int(0.5*cash/row['stock_close_usd']))
                if index+1 < len(merged_df):
                    adr_pos -= quantity
                    cash += quantity*row['adr_close']

                    stock_px = merged_df.loc[index+1,'stock_open_usd'] # The actual px we get to trade is on the next day for asian market
                    cash -= stock_px*quantity
                    stock_pos += quantity
                    trade_records.append("Opening positions:\n")
                    trade_records.append(f"We sold the {quantity} shares of ADR at the price of {row['adr_close']} on {row['date']}\n")
                    trade_records.append(f"We bought the {quantity} shares of underlying stock at the price of {stock_px} USD ({merged_df.loc[index+1,'stock_open']} Japanese dollars) on {merged_df.loc[index+1,'date']}\n")


        # When do we exit the position?
        elif row['adr_close'] - row['stock_close_usd'] < np.array(diff_record).mean():
            if stock_pos > 0 and adr_pos < 0 : # If we have positions in the stocks, we liquidate the position
                if index+1 < len(merged_df):
                
                    cash -= abs(adr_pos)*row['adr_close']
                    cash += stock_pos*merged_df.loc[index+1,'stock_open_usd']
                    
                    trade_records.append("Closing positions:\n")
                    trade_records.append(f"We bought the {stock_pos} shares of ADR at the price of {row['adr_close']} on {row['date']}\n")
                    trade_records.append(f"We sold the {stock_pos} shares of underlying stock at the price of {merged_df.loc[index+1,'stock_open_usd']} USD ({merged_df.loc[index+1,'stock_open']} Japanese dollars) on {merged_df.loc[index+1,'date']}\n")
                    stock_pos, adr_pos = 0, 0
        diff_record.append(row['adr_close'] - row['stock_close_usd'])
    final_val = cash + adr_pos*merged_df.loc[len(merged_df) - 1, 'adr_close'] + stock_pos*merged_df.loc[len(merged_df) - 1, 'stock_close_usd'] 
    ret = (final_val - starting_cash)/starting_cash
    
    
    return ret, trade_records

In [134]:
"""
Old code snippets

# Grab the csv data for the stocks
adr_path = 'eric_jh_data/Japan/IX_8591/adr.csv'
stock_path = 'eric_jh_data/Japan/IX_8591/underlying.csv'
fx_path = 'eric_jh_data/Forex/USD_JPY.csv'
adr_df = pd.read_csv(adr_path, index_col = 0).rename(columns = {'close':'adr_close', 'open':'adr_open'})
stock_df = pd.read_csv(stock_path, index_col = 0).rename(columns = {'close':'stock_close', 'open':'stock_open'})
fx_df = pd.read_csv(fx_path, index_col = 0).rename(columns = {'close':'fx_close', 'open':'fx_open'})

# Find the ratio between adr and stock:


merged_df = pd.merge(adr_df.loc[:,['date', 'adr_open','adr_close']], stock_df.loc[:,['date', 'stock_open','stock_close']])
merged_df = pd.merge(merged_df, fx_df.loc[:,['date', 'fx_open','fx_close']])
merged_df['stock_open_usd'] = merged_df['stock_open']/merged_df['fx_open']
merged_df['stock_close_usd'] = merged_df['stock_close']/merged_df['fx_close']



"""

110513.74525800598